# Diffie-Hellman & RSA Key Exchange 정리 및 실습

공무원 시험 및 보안기사 필기 대비 실무자용 Python 실습 노트북입니다.

## Diffie-Hellman Key Exchange

1. 공개소수 `p`, 원시근 `g`를 공유한다.
2. Alice는 비밀키 `a`, Bob은 비밀키 `b`를 고른다.
3. Alice는 공개키 `A = g^a mod p`, Bob은 `B = g^b mod p` 계산 후 교환
4. 공유 비밀키: `K = B^a mod p = A^b mod p`


In [1]:
# Diffie-Hellman 구현 예제
def diffie_hellman(p, g, a, b):
    A = pow(g, a, p)
    B = pow(g, b, p)
    shared_key_alice = pow(B, a, p)
    shared_key_bob = pow(A, b, p)
    return A, B, shared_key_alice, shared_key_bob

# 예시
p = 23  # 소수
g = 5   # 원시근
a = 6   # Alice의 비밀키
b = 15  # Bob의 비밀키

A, B, key_alice, key_bob = diffie_hellman(p, g, a, b)
print("Alice의 공개키 A:", A)
print("Bob의 공개키 B:", B)
print("Alice가 계산한 공유키:", key_alice)
print("Bob이 계산한 공유키:", key_bob)

Alice의 공개키 A: 8
Bob의 공개키 B: 19
Alice가 계산한 공유키: 2
Bob이 계산한 공유키: 2


## RSA Key Generation & Encryption/Decryption

1. 소수 `p`, `q` 선택
2. `n = p * q`, `phi = (p-1)*(q-1)` 계산
3. `1 < e < phi` 이면서 서로소인 `e` 선택
4. `d ≡ e⁻¹ mod phi` (e의 모듈러 역원) 계산
5. 공개키 `(e, n)`, 개인키 `(d, n)`
6. 암호화: `C = M^e mod n`, 복호화: `M = C^d mod n`


In [2]:
# RSA 구현 예제
from sympy import mod_inverse, isprime, gcd

def rsa_keygen(p, q, e):
    assert isprime(p) and isprime(q), "p, q는 소수여야 함"
    n = p * q
    phi = (p - 1) * (q - 1)
    assert gcd(e, phi) == 1, "e와 phi는 서로소여야 함"
    d = mod_inverse(e, phi)
    return (e, n), (d, n)

def rsa_encrypt(m, pubkey):
    e, n = pubkey
    return pow(m, e, n)

def rsa_decrypt(c, privkey):
    d, n = privkey
    return pow(c, d, n)

# 예시
p, q = 61, 53
e = 17
pubkey, privkey = rsa_keygen(p, q, e)
message = 65

cipher = rsa_encrypt(message, pubkey)
print("암호문:", cipher)
plain = rsa_decrypt(cipher, privkey)
print("복호화된 평문:", plain)

암호문: 2790
복호화된 평문: 65


## ECC (Elliptic Curve Cryptography) 개념 및 실습

- 타원곡선 위의 덧셈/곱셈 연산 사용
- E: y² = x³ + ax + b (mod p) 형태의 곡선 위의 점들을 이용
- 개인키: 정수 `k`, 공개키: `R = kP` (P는 공개된 곡선 위 점)
- 문제: `P`, `R`가 주어졌을 때 `k`를 찾는 것 → 이산로그 문제

👉 공무원/보안기사 시험에서 나오는 유형: `R = kP` 만족하는 `k`를 그래프 보고 찾기 (시각적 순환 구조)


In [3]:
# ECC 곡선 상의 점 덧셈 구현 (단순한 예시, 실제는 더 복잡함)
# y^2 = x^3 + ax + b over finite field mod p
def inverse_mod(k, p):
    return pow(k, -1, p)

def ecc_add(P, Q, a, p):
    if P == Q:
        l = (3 * P[0]**2 + a) * inverse_mod(2 * P[1], p)
    else:
        l = (Q[1] - P[1]) * inverse_mod(Q[0] - P[0], p)
    l %= p
    x_r = (l**2 - P[0] - Q[0]) % p
    y_r = (l * (P[0] - x_r) - P[1]) % p
    return (x_r, y_r)

def ecc_mul(P, k, a, p):
    R = P
    for _ in range(k - 1):
        R = ecc_add(R, P, a, p)
    return R

# 예시: E: y^2 = x^3 + 2x + 2 mod 17
a, b, p = 2, 2, 17
P = (5, 1)
k = 7
R = ecc_mul(P, k, a, p)
print(f"k = {k}일 때 R = kP = {R}")

k = 7일 때 R = kP = (0, 6)


## ElGamal 암호 (이산대수 문제 기반)

- 공개키: `(p, a, y)` (소수 p, 원시근 a, y = a^x mod p)
- 개인키: `x`
- 시험문제: `y = a^x mod p` 만족하는 x는? → 이산로그 문제
- p 작을 때는 bruteforce로 `x` 찾기 가능


In [4]:
# ElGamal 이산로그 문제
def discrete_log_brute(a, y, p):
    for x in range(1, p):
        if pow(a, x, p) == y:
            return x
    return None

# 예시
p = 23
a = 5
y = 4
x = discrete_log_brute(a, y, p)
print(f"a = {a}, y = {y}, p = {p}일 때, a^x ≡ y mod p 를 만족하는 x = {x}")

a = 5, y = 4, p = 23일 때, a^x ≡ y mod p 를 만족하는 x = 4
